# Introdução

### Notebook baseado em: [pierretihon notebook](https://www.kaggle.com/pierretihon/a-homemade-tensorflow-model).

#### Esse notebook tem o propósito de mostrar como construir uma rede convolucional usando Pytorch.
#### Iremos criar a rede, as classes necessárias para carregar os dados, ver como utilizar uma GPU para o treinamento, e muito mais!


#### O notebook está dividido em:

1. Módulos/Classes
    1. Config
    2. Dataset
    3. Modelo
2. Carregamento dos dados
3. Visualização dos dados
4. Coletando um subset dos dados
5. Treino do modelo
6. Submissão



# Importando classes necessárias

In [ ]:

import os
import pickle
from PIL import Image

# classes relativas ao PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms

from albumentations import (
    Compose, Normalize, Resize, RandomResizedCrop, RandomCrop, HorizontalFlip
)
from albumentations.pytorch import ToTensorV2

import cv2

import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split


Para garantir que a execução use a mesma seed aleatória, configuramos para ser a mesma em qualquer execução do notebook.

In [ ]:
seed = 42
np.random.seed(42)
torch.cuda.manual_seed_all(seed)

# 1) Módulos/Classes

## 1.1 Config class
Classe que contém alguns parâmetros de configuração, como tamanho da imagem, caminho (*path*) até a imagem, classes que queremos predizer.

In [ ]:
class Config:
    img_train_path = '../input/ranzcr-clip-catheter-line-classification/train'
    img_test_path = '../input/ranzcr-clip-catheter-line-classification/test'
    
    batch_size = 64
    img_width= 256
    img_height = 256
    
    target_cols = ['ETT - Abnormal', 'ETT - Borderline', 'ETT - Normal',
                   'NGT - Abnormal', 'NGT - Borderline', 'NGT - Incompletely Imaged', 'NGT - Normal',
                   'CVC - Abnormal', 'CVC - Borderline', 'CVC - Normal', 'Swan Ganz Catheter Present']
    
    n_classes = len(target_cols)
    n_workers = 8

## 1.2 Dataset class
Para carregarmos os dados para treinamento em Pytorch, é recomendado a construção de uma classe Dataset, que abstrai o carregamento dos dados, e permite que apliquemos transformações dado o contexto (treino, validação ou teste). Nela, é necessário que implementemos os seguintes métodos:
- __init__
- __getiitem__
- __len__

In [ ]:
class ImageDataset(Dataset):
    
    def __init__(self, df, mode):
        super().__init__()
        self.filenames = df['StudyInstanceUID'].values
        self.labels = df[Config.target_cols].values
        self.len = len(df)
        self.transform = self.train_transforms() if mode == 'train' else self.valid_transforms() if mode == 'valid' else None
        
    def __getitem__(self, idx):
        filename = self.filenames[idx]
        filepath = f'{Config.img_train_path}/{filename}.jpg'
        image = cv2.imread(filepath)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
        
        label = torch.tensor(self.labels[idx]).float()
        return image, label
    
    def __len__(self):
        return self.len
    
    def train_transforms(self):
        # transformações usadas no treino
        return Compose([
            Resize(Config.img_width, Config.img_height),
            RandomResizedCrop(Config.img_width, Config.img_height, scale=(0.85, 1.0)),
            HorizontalFlip(p=0.5),
            Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
            ToTensorV2(),
        ])

    def valid_transforms(self):
        # transformações usadas em validação
        return Compose([
            Resize(Config.img_width, Config.img_height),
            Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
            ToTensorV2(),
        ])

## 1.3 Model class
Agora temos de construir a classe do modelo. Em Pytorch, o módulo ```torch.nn``` contém as abstrações de uma rede neural necessárias para o aprendizado, como forward, ajuste dos pesos etc. Isso nos ajuda bastante, porque para definir uma rede, basta herdar o módulo em sua classe, e implementar 2 métodos:

- __init__
- __forward__

### Vamos a construção da rede abaixo:
Ela é uma rede neural convolucional bem simples com basicamente 3 peças: as convoluções, o average pooling ([mais informações](https://pytorch.org/docs/stable/generated/torch.nn.AvgPool2d.html)) e a última parte, uma rede densa.

O average pooling até não seria necessário, já que a parte convolucional e a densa já iriam compor uma rede neural convolucional.

#### Na parte convolucional
São 4 camadas encadeadas de: Convolução, função de ativação (ReLU) e max pooling.
Você pode encontrar mais informações sobre essas operações [aqui](https://cs231n.github.io/convolutional-networks/).

#### Na parte densa
São 5 camadas e uma de output, as 3 primeiras com [dropout](https://deeplearningbook.com.br/capitulo-23-como-funciona-o-dropout/), as duas últimas sem. Nossa saída é uma sigmóide, com 11 classes.

In [ ]:
class SimpleModel(nn.Module):
    def __init__(self, n_classes=Config.n_classes):
        super(SimpleModel, self).__init__()
        self.conv = nn.Sequential(
            # first
            nn.Conv2d(in_channels=3, out_channels=128, kernel_size=4, stride=4, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            # second
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=2, stride=2, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            # third
            nn.Conv2d(in_channels=256, out_channels=128, kernel_size=2, stride=2, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            # fourth
            nn.Conv2d(in_channels=128, out_channels=64, kernel_size=2, stride=2, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.avgpool = nn.AdaptiveAvgPool2d((4, 4))
        self.dense = nn.Sequential(
            nn.Flatten(),
            #first
            nn.Dropout(p=0.05),
            nn.Linear(64*4*4, 512),
            nn.ReLU(),
            # second
            nn.Dropout(p=0.1),
            nn.Linear(512, 512),
            nn.ReLU(),
            # third
            nn.Dropout(p=0.3),
            nn.Linear(512, 128),
            nn.ReLU(),
            # fourth
            nn.Linear(128, 64),
            nn.ReLU(),
            # fifth
            nn.Linear(64, 32),
            nn.ReLU(),
            # output
            nn.Linear(32, n_classes),
            nn.Sigmoid()
        )
        
    
    def forward(self, x):
        x = self.conv(x)
        x = self.avgpool(x)
        x = self.dense(x)
        return x

# 2) Carregamento dos dados

Iremos carregar os dados direto do Kaggle.

In [ ]:
# useful paths
catherer_path = '/kaggle/input/ranzcr-clip-catheter-line-classification'
train_path = os.path.join(catherer_path,'train')
test_path = os.path.join(catherer_path,'test')

Observando o shape do dataset e valores nulos:

In [ ]:
# train
train_csv_path = os.path.join(catherer_path,'train.csv')
train_df = pd.read_csv(train_csv_path)

classes = [col for col in train_df.columns if col not in ['StudyInstanceUID','PatientID']]
print(f'There are {len(classes)} to predict')

print(f"Shape of train dataframe : {train_df.shape}")
print(f"check for null values: {train_df.isnull().sum().sum()}")

# test
test_csv_path = os.path.join(catherer_path,'sample_submission.csv')
test_df = pd.read_csv(test_csv_path)
test_filenames = test_df.StudyInstanceUID

print(f"Shape of test dataframe : {test_df.shape}")
print(f"check for null values: {test_df.isnull().sum().sum()}")

Observando um trecho do nosso conjunto:

In [ ]:
train_df.head()

# 3) Visualização dos dados

Observando uma imagem:

In [ ]:
img_path = Config.img_train_path + '/' + train_df['StudyInstanceUID'].iloc[0] + '.jpg'
img_example = Image.open(img_path)
print(f"Image size = {img_example.size}")
plt.figure(figsize=(12,8))
plt.imshow(img_example,cmap='Greys');

Podemos verificar pelo shape da imagem que ela é grande, e podemos diminuir as dimensões dela sem muita perda de informação.

In [ ]:
img_example_red = img_example.resize((Config.img_width, Config.img_height))
plt.figure(figsize=(12,8))
plt.imshow(img_example_red,cmap='Greys');

# 4) Coletando um subset dos dados

Podemos limitar a quantidade de dados utilizadas para treino, para acelerar o processo de verificar o comportamento do modelo:

In [ ]:
frac = 0.8
lim = True
if lim:
    red_train_df = train_df.sample(frac=frac)
else:
    red_train_df = train_df.copy()
print(red_train_df.shape)

In [ ]:
n_min = 1000
count_classes = red_train_df[classes].sum()
ext_train_df = [red_train_df]
for pred_class in classes:
    if count_classes[pred_class] < n_min:
        new_df = red_train_df[red_train_df[pred_class]==1].sample(n_min,replace=True)
        ext_train_df.append(new_df)
        
ext_train_df = pd.concat(ext_train_df)
print(ext_train_df.shape)

Agora o número de classes em nosso DataFrame é:

In [ ]:
plt.figure(figsize=(10,6))
graph = sns.barplot(x=classes,y=ext_train_df[classes].sum())
graph.set_xticklabels(graph.get_xticklabels(), rotation=90);

# 5) Treino do modelo

In [ ]:
# Dividindo em treino e validação
X_train, X_valid = train_test_split(ext_train_df, test_size=0.2, shuffle=True)
print(f' X_train shape: {X_train.shape} and X_valid shape: {X_valid.shape}')

In [ ]:
train_dataset = ImageDataset(df=X_train, mode='train')
valid_dataset = ImageDataset(df=X_valid, mode='valid')
test_dataset = ImageDataset(df=test_df, mode='valid')

# A classe DataLoader é necessária para o carregamento dos batches
train_dataloader = DataLoader(train_dataset, pin_memory=True, batch_size=Config.batch_size, num_workers=Config.n_workers, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, pin_memory=True, batch_size=Config.batch_size, num_workers=Config.n_workers, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=Config.batch_size, num_workers=Config.n_workers, shuffle=False)

In [ ]:
model = SimpleModel()
# Nosso otimizador, necessário para o treinamento da rede, junto com a função de custo
optimizer = torch.optim.AdamW(model.parameters())
loss = nn.BCELoss()

### Treinamento da rede

Abaixo, temos a função com o loop de treinamento da nossa rede. 

1. Nele, fazemos um loop no dataloader, recuperando os dados de treino e o target. Aqui é necessário chamar o método .to(device) para passarmos os dados a GPU;
2. Zeramos os gradientes previamente computados (necessário no Pytorch);
3. Dizemos que o modelo está em modo de treinamento (```model.train()```);
4. Passamos a imagem ao modelo (convertendo os valores para float) e temos o output;
5. O output é usado para calcularmos a loss, utilizando também nossos valores de target;
6. Realizamos o backward propagation;
7. Damos um passo em direção a um mínimo local com o otimizador.

Ufa! Foram muitas etapas, mas são elas que possibilitam que nossa rede aprenda.
Também, ao fim do loop de treino, chamamos o função de validação para verificar como a função de custo se comporta.

In [ ]:
train_history = []
valid_history = []

def train(model, loss, optimizer, train_dataloader, valid_dataloader, device='cuda', epochs=50):
    for e in range(1, epochs+1):
        for train_values, train_target in train_dataloader:
            train_values, train_target = train_values.to(device), train_target.to(device)
            # loop principal
            optimizer.zero_grad()   
            model.train()
            output = model(train_values.float())
            loss_train = loss(output, train_target.float())
            loss_train.backward()
            optimizer.step()
        
        print(f'Epoch {e}: \ttrain loss {loss_train.item():.2f}')
        valid(model, loss, optimizer, valid_dataloader, device, epochs=1)
        train_history.append(loss_train)
            
def valid(model, loss, optimizer, valid_dataloader, device, epochs=50):
    for e in range(1, epochs+1):
        for val_values, val_target in valid_dataloader:
            val_values, val_target = val_values.to(device), val_target.to(device)
            
            model.eval()
            val_output = model(val_values.float())
            loss_val = loss(val_output, val_target.float())
        
        valid_history.append(loss_val)
        print(f'\t\tvalidation loss {loss_val.item():.2f}')
            
def predict_probs(filenames, model, device='cuda'):
    model.eval()

    transform = test_dataset.valid_transforms()
    
    predictions = []
    for filename in tqdm(filenames):
        filepath = f'{Config.img_test_path}/{filename}.jpg'
        image = cv2.imread(filepath)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = transform(image=image)['image']
        image = image.unsqueeze(0).to(device)
        predictions.append(model(image).detach().cpu().numpy())
    return predictions

### Um ponto crucial para treinarmos a rede convolucional é o uso da GPU

Precisamos verificar se a GPU está disponível, e posteriormente, passar o modelo para GPU (para que ele seja processado nessa unidade, e não na CPU)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(f'Current device is {device}')
print(model)

Treinando o modelo:

In [ ]:
# Treinamento do modelo
train(model, loss, optimizer, train_dataloader, valid_dataloader, device, epochs=15)

# 6) Submissão

Para não precisarmos rodar o notebook todas as vezes para obter as saídas do modelo, podemos salvar os pesos da rede.
Em torch, a função save permita que façamos isso:

In [ ]:
torch.save(model.state_dict(), 'simple_model.pt')

Verificando a curva de treino e de validação:

In [ ]:
plt.plot(range(len(train_history)), train_history)
plt.plot(range(len(valid_history)), valid_history)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.show();

### Fazendo as predições no conjunto de teste e salvando em formato pickle

In [ ]:
predictions = predict_probs(test_filenames, model)
print(f'Predictions type = {type(predictions)}')
print(f'Predictions size = {len(predictions)}')
with open("preds.pkl", "wb") as fp:
    pickle.dump(predictions, fp)

In [ ]:
# Carrega o arquivo pkl
# with open('../input/preds2/preds.pkl', 'rb') as pickle_file:
#     preds = pickle.load(pickle_file)

In [ ]:
# Exemplo de predição
print(predictions[0])

In [ ]:
preds_one_len = []
for pred in predictions:
    preds_one_len.append(pred.squeeze())

### Criamos um DataFrame para submeter nossas predições, com os respectivos nomes das categorias

In [ ]:
pred_df = pd.DataFrame(columns=Config.target_cols, data=preds_one_len, index=test_df.index)
pred_df = pd.concat([test_df['StudyInstanceUID'],pred_df],axis=1)
pred_df

In [ ]:
# Create the csv
pred_df.to_csv('submission.csv',index=False)